<a href="https://colab.research.google.com/github/VictorMadu/Fraud-Detection-Challenge/blob/master/knn_distance.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib
import os
from scipy import stats
import pickle
import warnings; warnings.filterwarnings('ignore')

In [0]:
df = pd.read_csv("/content/drive/My Drive/fraud detection/data/processed_train.csv")

In [0]:
print(df.shape[0], df.shape[1])

10493 1259


In [0]:
df.head()

,account destination verification,account source verification,account type,age,credit card type,current bank amount,fradulent,id,last bank amount,marital_status,most recent bank amount,occupation,time taken (seconds),transaction source method,is_busday,year,quarter,month,week,day,hour,minute,second,weekday,no_days_in_month,age_group,diff_current_and_last_amount,diff_recent_and_last_amount,diff_recent_and_current_amount,account type_current bank amount_mean,account type_current bank amount_max,account type_current bank amount_min,account type_current bank amount_range,account type_current bank amount_sum,account type_current bank amount_skew,account type_current bank amount_kurtosis,account type_current bank amount_std,account type_current bank amount_count,account type_last bank amount_mean,account type_last bank amount_max,...,no_days_in_month_age_min,no_days_in_month_age_range,no_days_in_month_age_sum,no_days_in_month_age_skew,no_days_in_month_age_kurtosis,no_days_in_month_age_std,no_days_in_month_age_count,no_days_in_month_diff_current_and_last_amount_mean,no_days_in_month_diff_current_and_last_amount_max,no_days_in_month_diff_current_and_last_amount_min,no_days_in_month_diff_current_and_last_amount_range,no_days_in_month_diff_current_and_last_amount_sum,no_days_in_month_diff_current_and_last_amount_skew,no_days_in_month_diff_current_and_last_amount_kurtosis,no_days_in_month_diff_current_and_last_amount_std,no_days_in_month_diff_current_and_last_amount_count,no_days_in_month_diff_recent_and_last_amount_mean,no_days_in_month_diff_recent_and_last_amount_max,no_days_in_month_diff_recent_and_last_amount_min,no_days_in_month_diff_recent_and_last_amount_range,no_days_in_month_diff_recent_and_last_amount_sum,no_days_in_month_diff_recent_and_last_amount_skew,no_days_in_month_diff_recent_and_last_amount_kurtosis,no_days_in_month_diff_recent_and_last_amount_std,no_days_in_month_diff_recent_and_last_amount_count,no_days_in_month_diff_recent_and_current_amount_mean,no_days_in_month_diff_recent_and_current_amount_max,no_days_in_month_diff_recent_and_current_amount_min,no_days_in_month_diff_recent_and_current_amount_range,no_days_in_month_diff_recent_and_current_amount_sum,no_days_in_month_diff_recent_and_current_amount_skew,no_days_in_month_diff_recent_and_current_amount_kurtosis,no_days_in_month_diff_recent_and_current_amount_std,no_days_in_month_diff_recent_and_current_amount_count,current bank amount/time taken (seconds),last bank amount/timetime taken (seconds),most recent bank amount/time taken (seconds),diff_current_and_last_amount/time taken (seconds),diff_recent_and_last_amount/time taken (seconds),diff_recent_and_current_amount/time taken (seconds)
0,1,1,saving,46,master,43671,0,acus0,1151263,married,348764,musician,2299,0,1,2016,4,10,40,5,1,44,58,2,31,4,-1107592,-802499,305093,49890.026256,99917,30,99887,110207068,-0.001333,-1.197369,29083.038329,2209,751193.550023,1499877,...,30,49,473950,0.000565,-1.197589,14.448248,8677,-696990.075141,99489,-1494270,1593759,-6047782882,-0.016360,-1.194548,435851.205717,8677,-498999.923822,483791,-1489198,1972989,-4329822339,-0.007437,-0.987293,457822.152474,8677,197990.151320,497672,-94720,592392,1717960543,0.012826,-1.103505,146751.567276,8677,18.995650,500.766855,151.702479,-481.771205,-349.064376,305093
1,0,1,current,64,master,7442,0,acus1,1062843,single,225114,accounting,4553,0,1,2015,1,1,3,14,15,5,7,2,31,7,-1055401,-837729,217672,50017.612775,99996,1,99995,639775285,-0.004401,-1.193485,28751.805897,12791,750144.725119,1499722,...,30,49,473950,0.000565,-1.197589,14.448248,8677,-696990.075141,99489,-1494270,1593759,-6047782882,-0.016360,-1.194548,435851.205717,8677,-498999.923822,483791,-1489198,1972989,-4329822339,-0.007437,-0.987293,457822.152474,8677,197990.151320,497672,-94720,592392,1717960543,0.012826,-1.103505,146751.567276,8677,1.634527,233.437953,49.443005,-231.803426,-183.994948,217672
2,1,1,current,41,master,40102,0,acus2,48042,married,420486,farmer,1493,1,1,2016,4,10,43,28,13

In [0]:
df[df.isnull().any(1)]

,account destination verification,account source verification,account type,age,credit card type,current bank amount,fradulent,id,last bank amount,marital_status,most recent bank amount,occupation,time taken (seconds),transaction source method,is_busday,year,quarter,month,week,day,hour,minute,second,weekday,no_days_in_month,age_group,diff_current_and_last_amount,diff_recent_and_last_amount,diff_recent_and_current_amount,account type_current bank amount_mean,account type_current bank amount_max,account type_current bank amount_min,account type_current bank amount_range,account type_current bank amount_sum,account type_current bank amount_skew,account type_current bank amount_kurtosis,account type_current bank amount_std,account type_current bank amount_count,account type_last bank amount_mean,account type_last bank amount_max,...,no_days_in_month_age_min,no_days_in_month_age_range,no_days_in_month_age_sum,no_days_in_month_age_skew,no_days_in_month_age_kurtosis,no_days_in_month_age_std,no_days_in_month_age_count,no_days_in_month_diff_current_and_last_amount_mean,no_days_in_month_diff_current_and_last_amount_max,no_days_in_month_diff_current_and_last_amount_min,no_days_in_month_diff_current_and_last_amount_range,no_days_in_month_diff_current_and_last_amount_sum,no_days_in_month_diff_current_and_last_amount_skew,no_days_in_month_diff_current_and_last_amount_kurtosis,no_days_in_month_diff_current_and_last_amount_std,no_days_in_month_diff_current_and_last_amount_count,no_days_in_month_diff_recent_and_last_amount_mean,no_days_in_month_diff_recent_and_last_amount_max,no_days_in_month_diff_recent_and_last_amount_min,no_days_in_month_diff_recent_and_last_amount_range,no_days_in_month_diff_recent_and_last_amount_sum,no_days_in_month_diff_recent_and_last_amount_skew,no_days_in_month_diff_recent_and_last_amount_kurtosis,no_days_in_month_diff_recent_and_last_amount_std,no_days_in_month_diff_recent_and_last_amount_count,no_days_in_month_diff_recent_and_current_amount_mean,no_days_in_month_diff_recent_and_current_amount_max,no_days_in_month_diff_recent_and_current_amount_min,no_days_in_month_diff_recent_and_current_amount_range,no_days_in_month_diff_recent_and_current_amount_sum,no_days_in_month_diff_recent_and_current_amount_skew,no_days_in_month_diff_recent_and_current_amount_kurtosis,no_days_in_month_diff_recent_and_current_amount_std,no_days_in_month_diff_recent_and_current_amount_count,current bank amount/time taken (seconds),last bank amount/timetime taken (seconds),most recent bank amount/time taken (seconds),diff_current_and_last_amount/time taken (seconds),diff_recent_and_last_amount/time taken (seconds),diff_recent_and_current_amount/time taken (seconds)


In [0]:
cat = ['age', 'account type','account source verification', 'transaction source method', 'account destination verification', 'credit card type',  'age_group', 'marital_status', 'occupation', 'is_busday', 'year', 'quarter', 'month', 'week', 'day', 'hour', 'weekday', 'no_days_in_month', 'minute', 'second']
for i in cat:
  df[i] = df[i].astype('category')

In [0]:
gat =  ['age', 'account type', 'account source verification', 'transaction source method', 'account destination verification', 'credit card type', 'age_group', 'marital_status', 'occupation', 'is_busday', 'year', 'quarter', 'month', 'week', 'day', 'hour', 'weekday', 'no_days_in_month', 'minute', 'second', 'fradulent']
gf = df[gat]
cols_dict = dict()                                 
for col in cat:
  groups = gf.groupby(col)['fradulent'].mean()
  uniques = gf[col].unique()
  uniques_dict = dict(zip(uniques, groups))
  cols_dict[col] = uniques_dict

In [0]:
gf.head()

,age,account type,account source verification,transaction source method,account destination verification,credit card type,age_group,marital_status,occupation,is_busday,year,quarter,month,week,day,hour,weekday,no_days_in_month,minute,second,fradulent
0,46,saving,1,0,1,master,4,married,musician,1,2016,4,10,40,5,1,2,31,44,58,0
1,64,current,1,0,0,master,7,single,accounting,1,2015,1,1,3,14,15,2,31,5,7,0
2,41,current,1,1,1,master,3,married,farmer,1,2016,4,10,43,28,13,4,31,44,58,0
3,66,current,1,1,1,master,8,unknown,clergy,0,2016,4,11,46,20,15,6,30,5,32,0
4,78,current,0,0,1,verve,10,married,accounting,1,2015,3,9,36,1,16,1,30,11,57,1


In [0]:
from sklearn.base import BaseEstimator, TransformerMixin, clone
class MeanEncoder(BaseEstimator, TransformerMixin):
    
    def __init__(self, cols=[]):
        self.cols = cols 

    def transform(self, X, **transformparams):
        assert isinstance(X, pd.DataFrame) 
        for col in self.cols:
            X[col+" mean_encoded"] = X[col].map(self.mean_mapped_dict[col])
        return X

        
    def fit(self, X, y=None, **fitparams):
      assert isinstance(X, pd.DataFrame)
      global cols_dict 
      self.mean_mapped_dict = cols_dict.copy()
      return self

In [0]:
from sklearn.utils import resample


df_majority = df[df.fradulent==0]
df_minority = df[df.fradulent==1]
 
# Upsample minority class
df_minority_upsampled = resample(df_minority, 
                                 replace=True,     # sample with replacement
                                 n_samples=len(df_majority),    # to match majority class
                                 random_state=32) # reproducible results
 
# Combine majority class with upsampled minority class
df_upsampled = pd.concat([df_majority, df_minority_upsampled])


X = df_upsampled.drop(['fradulent', 'id'], axis=1)
y = df_upsampled['fradulent']

df_minority_upsampled.shape

(7590, 1259)

In [0]:
from sklearn.model_selection import train_test_split, cross_val_score, RandomizedSearchCV
from scipy.stats import uniform, truncnorm, randint

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [0]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder, PolynomialFeatures, OrdinalEncoder

cat_feat = df.drop(['fradulent','id'], axis=1).select_dtypes(exclude=['int', 'float']).columns

num_transform1 = Pipeline(steps=[
    ('scaler', StandardScaler()),
    ('poly', PolynomialFeatures(degree=2, include_bias=False))
])

num_transform2 = Pipeline(steps=[
    ('scaler', StandardScaler())
])


cat_transform1 = Pipeline(steps=[
    ('mean_encoder', MeanEncoder(cols=cat_feat)),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
    
])

cat_transform2 = Pipeline(steps=[
    ('mean_encoder', MeanEncoder(cols=cat_feat)),
    ('ordinal', OrdinalEncoder())
    
])


In [0]:
from sklearn.compose import ColumnTransformer
from scipy.stats import uniform, truncnorm, randint
from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.feature_selection import SelectKBest


num_feat = X.select_dtypes(include=['int', 'float']).columns
cat_feat = X.select_dtypes(exclude=['int', 'float']).columns

preprocesser = ColumnTransformer(transformers=[
      ('num', num_transform2, num_feat),
      ('cat', cat_transform1, cat_feat)
  ])



clf_pipe = Pipeline(steps=[
      ('preprocesser', preprocesser),
      ('fs', SelectKBest(k=310)),
      ('classifier', KNeighborsClassifier(n_neighbors=10, weights='uniform'))
  ])



print('Model training')
clf = clf_pipe.fit(X, y)

print('Model finished training')
# print('\nBest score:', lr.best_score_)
# print('\nBest Params:', lr.best_params_)
y_pred = clf.predict(X)

print('\naccuarcy score', accuracy_score(y, y_pred))
print('\nroc auc score', roc_auc_score(y, y_pred))
  
if not os.path.exists("/content/drive/My Drive/fraud detection/models"):
    os.makedirs("/content/drive/My Drive/fraud detection/models")

pickle.dump(clf, open(f"/content/drive/My Drive/fraud detection/models/knn_uniform", "wb"))
print("_"*70)
  

Model training
Model finished training

accuarcy score 0.8204874835309618

roc auc score 0.8204874835309618
______________________________________________________________________


In [0]:
help(cross_val_score)

Help on function cross_val_score in module sklearn.model_selection._validation:

cross_val_score(estimator, X, y=None, groups=None, scoring=None, cv=None, n_jobs=None, verbose=0, fit_params=None, pre_dispatch='2*n_jobs', error_score=nan)
    Evaluate a score by cross-validation
    
    Read more in the :ref:`User Guide <cross_validation>`.
    
    Parameters
    ----------
    estimator : estimator object implementing 'fit'
        The object to use to fit the data.
    
    X : array-like
        The data to fit. Can be for example a list, or an array.
    
    y : array-like, optional, default: None
        The target variable to try to predict in the case of
        supervised learning.
    
    groups : array-like, with shape (n_samples,), optional
        Group labels for the samples used while splitting the dataset into
        train/test set. Only used in conjunction with a "Group" :term:`cv`
        instance (e.g., :class:`GroupKFold`).
    
    scoring : string, callable or 